In [84]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # initialize weights matrix and biases
        self.W_input_hidden = np.random.rand(self.input_size, self.hidden_size)
        self.b_input_hidden = np.zeros((1, self.hidden_size))
        self.W_hidden_output = np.random.rand(self.hidden_size, self.output_size)
        self.b_hidden_output = np.zeros((1, self.output_size))

        # auxiliar functions

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def d_sigmoid(self, x):
        return x * (1 - x)

    def forward(self, input_data):
        hidden_layer_input = np.dot(input_data, self.W_input_hidden) + self.b_input_hidden
        hidden_layer_output = self.sigmoid(hidden_layer_input)
        output_layer_input = np.dot(hidden_layer_output, self.W_hidden_output) + self.b_hidden_output
        output = self.sigmoid(output_layer_input)

        return hidden_layer_output, output

    def backward(self, input_data, target, hidden_output, output, lr=0.2):

        # Backward propagation
        output_error = target - output
        output_grad = output_error * self.d_sigmoid(output)

        hidden_error = np.dot(output_grad, self.W_hidden_output.T)
        hidden_grad = hidden_error * self.d_sigmoid(hidden_output)

        # Update weights and biases using gradient descent
        self.W_hidden_output = self.W_hidden_output + np.dot(hidden_output.T, output_grad)*lr
        self.b_hidden_output = self.b_hidden_output + np.sum(output_grad, axis=0, keepdims=True)*lr

        self.W_input_hidden = self.W_input_hidden + np.dot(input_data.T, hidden_grad)*lr
        self.b_input_hidden = self.b_input_hidden + np.sum(hidden_grad, axis=0, keepdims=True)*lr

    def train(self, input_data, target, epochs=1000, lr=0.25):
        for epoch in range(epochs):
            hidden_output, output = self.forward(input_data)
            self.backward(input_data, target, hidden_output, output, lr=lr)
            if epoch % 1000 == 0:
                loss = np.mean((target - output) ** 2)
                print(f"Epoch {epoch}, Loss: {loss:.6f}")
    
        # Calculate R² (aggregated)
        sum_os_res = np.sum((target - output) ** 2)
        target_mean = np.mean(target, axis=0)  # Mean per output dimension
        sum_os_tot = np.sum((target - target_mean) ** 2)
        r2_aggregated = 1 - (sum_os_res / (sum_os_tot + 1e-8))
    
        # Calculate R² per output (optional)
        r2_per_output = []
        for i in range(target.shape[1]):  # Loop over each output dimension
            ss_res = np.sum((target[:, i] - output[:, i]) ** 2)
            ss_tot = np.sum((target[:, i] - np.mean(target[:, i])) ** 2)
            r2_i = 1 - (ss_res / (ss_tot + 1e-8))
            r2_per_output.append(r2_i)
    
        print(f"R² per output: {r2_per_output}")
        print(f"Average R²: {np.mean(r2_per_output):.6f}")
        return r2_aggregated




input_size = 4
hidden_size = 11
output_size = 3


# Load the Excel file
df = pd.read_excel("Parameters_AveragedLengthWidthArea_Modified.xlsx")

X = df[['Power', 'Gas', 'Tangential', 'Axial']]  
y = df[['Flame Width AveragePixel', 'Flame Length Average Pixel', 'Flame Area Average Pixel^2']]


X = X.to_numpy()
y = y.to_numpy()

# Shuffle
X, y = shuffle(X, y, random_state=42)

# Initialize scalers
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)


model = MLP(input_size, hidden_size, output_size)

print(model.train(X_scaled, y_scaled))




Epoch 0, Loss: 0.346694
R² per output: [0.7605762819954685, 0.9101106010411624, 0.7712551827712226]
Average R²: 0.813981
0.8204776118271896
